# <center> TP 4: Fisher Analysis <br>Linear Discriminant Analysis<br> <small>Réda DEHAK<br> 7 December 2018</small> </center>

The goal of this lab is :
    - Test the Fisher Analysis
    
We will use the [optdigits dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/) from UCI. This database is a small set of handwritten digits compared to the MNIST databases. A preprocessing program available by NIST was used to extract 64 features which represent the number of on pixel of nonoverlapping blocks of 4x4 extracted from normalized image. [See](optdigits.names.txt) for more details.

In [ ]:
import pandas as pd
import numpy as np

dataTrain = pd.read_csv('optdigits.tra', header=None)
dataTest = pd.read_csv('optdigits.tes', header=None)

Xtrain = dataTrain.values[:,0:64]
ytrain = dataTrain.values[:,64]

Xtest = dataTest.values[:,0:64]
ytest = dataTest.values[:,64]

1- Try to use a Principal Component Analysis of the training data

2- Plot each example of the training data in the first two factorial axis of PCA(2D space), use different color foreach classe? what did you conclude?

3- Train a logistic regression on the projected data into the $d$ first factorial axis of PCA, and test the results on the testing data? try different values of $d$?    

4- We will try a Linear Discriminant Analysis method to separate the 10 classes. Train a Linear Discriminant Analysis on the training data and plot each example in the 2d plan corresponding to the two axis with largest eigen values? what did you conclude?   

5- Train a logistic regression on the projected data into the $d$ first axis of LDA? what did you remark and how to solve this problem?

6- Test the results on the testing data? try different values of $𝑑$? what did you conclude?

7- Do the same task with the MNIST dataset?